# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Your code here

df = pd.read_csv('homepage_actions.csv')

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [6]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [48]:
c_unique = set(df[df['action'] == 'click']['id'].unique())
v_unique = set(df[df['action'] == 'view']['id'].unique())


print("Number of viewers: {} \tNumber of clickers: {}".format(len(v_unique), len(c_unique)))
print("Number of Viewers who didn't click: {}".format(len(v_unique-c_unique)))
print("Number of Clockers who didn't view: {}".format(len(c_unique-v_unique)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clockers who didn't view: 0


In [20]:
# idcounts_s = df.id.value_counts()

In [23]:
# repeat_ID_s = idcounts_s[idcounts_s > 1]
# print("{} repeat IDs".format(len(repeat_ID_s)))

1860 repeat IDs


In [31]:
# repeat_ID = list(repeat_ID_s.index)
# type(repeat_ID)

list

In [32]:
# df[df['id'].isin(repeat_ID)]

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
...,...,...,...,...
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view


In [17]:
exp_df = df[df['group']=='experiment']
cont_df = df[df['group']=='control']

print('total length:', len(df))
print('exp length:', len(exp_df))
print('cont length:', len(cont_df))

print('Overlap:', len(exp_df) + len(cont_df) - len(df))

total length: 8188
exp length: 3924
cont length: 4264
Overlap: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

Null: The experimental webpage was no more or less likely to attract clicks than the control

Alt: The experimental webpage was more or less likely to attract clicks than the control

In [65]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [68]:
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()
# experiment.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [57]:
# experiment_clicks = len(df[(df['action']=='click') & (df['group']=='experiment')])
# control_clicks = len(df[(df['action']=='click') & (df['group']=='control')])

In [70]:
experiment_total = len(experiment)
experiment_clicks = experiment.click.sum()

control_total = len(control)
control_clicks = control.click.sum()

experiment_noclicks = experiment_total - experiment_click
control_noclicks = control_total - control_click

In [71]:
contingency_table = np.array([
    (experiment_clicks, experiment_noclicks),
    (control_clicks, control_noclicks)
])

contingency_table

array([[ 928., 2068.],
       [ 932., 2400.]])

In [72]:
stats.chi2_contingency(contingency_table)

(6.712921132285344,
 0.00957168049704227,
 1,
 array([[ 880.61946903, 2115.38053097],
        [ 979.38053097, 2352.61946903]]))

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [74]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [75]:
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [76]:
#Your code here
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


In [77]:
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

It is smaller than what I originally calculated but both show a significant result (rejecting the null hypothesis)

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.